In [3]:
# Crawl


# import requests
# from bs4 import BeautifulSoup

# url = 'https://oec.world/en/product-landing/hs'
# reqs = requests.get(url)
# soup = BeautifulSoup(reqs.text, 'html.parser')
# print(soup.prettify())

# urls = []
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [4]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>')

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext+"\n"

def find_HSCODE(txt):
    found_list_temp = re.findall('HS Code: \d+\n', txt)
    found_list = []
    for i in found_list_temp:
        found_list += re.findall('\d+', i)
    return found_list

In [11]:
found_list = []

with open("OEC Products by Harmonized System Classification.txt", "r", encoding="utf-8") as f:
    text = f.read()
    found_list = find_HSCODE(text)
    
found_set = set()
for x in found_list:
    found_set.add(x)

"6281420" not in found_set
len(found_set)

7932

In [9]:
import json
import requests
from bs4 import BeautifulSoup

counter = 0

def write_product(data, f):
    for record in data["c"]:
        if (str(record["i"]) not in found_set):
            url = 'https://oec.world/en/profile/hs/'+record["s"]

            reqs = requests.get(url)
            soup = BeautifulSoup(reqs.text, 'lxml')

            name = '{0} HS Code: {1}\n'.format(record["cn"], record["i"])
            f.write(name)

            global counter
            counter += 1
            if (counter % 10 == 0):
                print(counter)
                print("We now at HS Code: ", record["i"])
            if (counter == 1):
                print(counter)
                print("We now at HS Code: ", record["i"])

            for paragraph in soup.find_all('p'):
                paragraph = str(paragraph)
                cleaned_paragraph = cleanhtml(paragraph)
                if "<strong>Overview" in paragraph:
                    f.write(cleaned_paragraph)
                if "<strong>Exports" in paragraph:
                    f.write(cleaned_paragraph)
                if "<strong>Imports" in paragraph:
                    f.write(cleaned_paragraph)
                if "<strong>Tariffs" in paragraph:
                    f.write(cleaned_paragraph)
                if "<strong>Ranking" in paragraph:
                    f.write(cleaned_paragraph)
                if "<strong>Description" in paragraph:
                    f.write(cleaned_paragraph)

            reqs = None
            soup = None
            f.write("\n")

        if (record["d"] < 3):
            write_product(record, f)

f = open("OEC Products by Harmonized System Classification.txt", "a", encoding="utf-8")

with open("OEC Product Metadata JSON.txt", "r") as jsonfile:
    data = json.load(jsonfile)
    
    write_product(data, f)

f.close()

1
We now at HS Code:  16845510
10
We now at HS Code:  16845620
20
We now at HS Code:  16845730
30
We now at HS Code:  16845931
40
We now at HS Code:  168460
50
We now at HS Code:  16846039
60
We now at HS Code:  168462
70
We now at HS Code:  168463
80
We now at HS Code:  16846510
90
We now at HS Code:  16846610
100
We now at HS Code:  16846721
110
We now at HS Code:  16846820
120
We now at HS Code:  16846929
130
We now at HS Code:  16847050
140
We now at HS Code:  16847170
150
We now at HS Code:  16847230
160
We now at HS Code:  16847410
170
We now at HS Code:  16847521
180
We now at HS Code:  16847690
190
We now at HS Code:  168478
200
We now at HS Code:  16847971
210
We now at HS Code:  16848041
220
We now at HS Code:  16848140
230
We now at HS Code:  16848291
240
We now at HS Code:  168484
250
We now at HS Code:  16848630
260
We now at HS Code:  16850131
270
We now at HS Code:  16850163
280
We now at HS Code:  16850240
290
We now at HS Code:  16850433
300
We now at HS Code:  1685059

In [14]:
import json
import requests
from bs4 import BeautifulSoup

counter = 0

def count_product(data):
    for record in data["c"]:
        global counter
        counter += 1

        if (record["d"] < 3):
            count_product(record)

with open("OEC Product Metadata JSON.txt", "r") as jsonfile:
    data = json.load(jsonfile)
    
    count_product(data)
    print(counter)

7932


In [3]:
# import requests
# from bs4 import BeautifulSoup

# with open("OEC Products by Harmonized System Classification.txt", "a", encoding="utf-8") as f:
#     url = "https://oec.world/en/profile/hs/ammonia-in-aqueous-solution"

#     reqs = requests.get(url)
#     soup = BeautifulSoup(reqs.text, 'html.parser')

#     for paragraph in soup.find_all('p'):
#         paragraph = str(paragraph)
#         if "<strong>Overview" in paragraph:
#             try:
#                 f.write(cleanhtml(paragraph))
#             except UnicodeEncodeError:
#                 print(url)
#         if "<strong>Exports" in paragraph:
#             f.write(cleanhtml(paragraph))
#         if "<strong>Imports" in paragraph:
#             f.write(cleanhtml(paragraph))
#         if "<strong>Tariffs" in paragraph:
#             f.write(cleanhtml(paragraph))
#         if "<strong>Ranking" in paragraph:
#             f.write(cleanhtml(paragraph))
#         if "<strong>Description" in paragraph:
#             try:
#                 f.write(cleanhtml(paragraph))
#             except UnicodeEncodeError:
#                 print(url)